In [12]:
# plot_igmas_moho_clean.py
# -*- coding: utf-8 -*-
"""
功能：
1. 读取 IGMAS 导出的 Moho CSV（x, y, z，单位 km，z 为深度，向下为负）。
2. 对 z 做简单清洗：去掉 0 / -80 等明显无效值，以及不在合理深度范围的点。
3. 对 (x, y) 相同的多条记录取中位数，消除重复点的小数抖动。
4. 将 x/y (km) 反投影为 lon/lat (deg)。
5. 采用“散点→规则网格插值(线性+最近邻补齐)”绘制 Moho 深度图，避免 tricontourf 在边界留白。

依赖：
- numpy
- pandas
- matplotlib
- scipy（可选；若无则退回 tricontourf，边界仍可能留白）
"""

from pathlib import Path
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    from scipy.interpolate import griddata
    _HAVE_SCIPY = True
except Exception:
    _HAVE_SCIPY = False

import matplotlib.tri as mtri


# ========= 配置区：按你的路径和区域修改 =========
IGMAS_XYZ_PATH = r"E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\moho_results.csv"
OUT_FIG_PATH   = r"E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\IGMAS_Moho_clean.png"

# 研究区范围
LON_MIN = 111.0
LON_MAX = 117.0
LAT_MIN = 14.5
LAT_MAX = 18.5

# 关键修改：用区域中纬度做经度方向换算（不要用 -2°）
LAT_REF = 0.5 * (LAT_MIN + LAT_MAX)

DEG2KM_LAT = 111.2
DEG2KM_LON = DEG2KM_LAT * math.cos(math.radians(LAT_REF))

# 绘图用规则网格分辨率（越大越平滑，但更慢）
GRID_NX = 360
GRID_NY = 360


# ========= 工具函数 =========
def read_igmas_xyz(path: Path) -> pd.DataFrame:
    """
    更稳健地读取 IGMAS xyz：
    - 优先按空白分隔读取（你的文件就是这种）
    - 若失败，再尝试逗号分隔
    """
    try:
        df = pd.read_csv(path, sep=r"\s+", comment="#", engine="python")
        if {"x", "y", "z"}.issubset(df.columns):
            return df[["x", "y", "z"]]
        if df.shape[1] >= 3:
            df = df.iloc[:, :3].copy()
            df.columns = ["x", "y", "z"]
            return df
    except Exception:
        pass

    df = pd.read_csv(path, comment="#")
    if {"x", "y", "z"}.issubset(df.columns):
        return df[["x", "y", "z"]]
    if df.shape[1] >= 3:
        df = df.iloc[:, :3].copy()
        df.columns = ["x", "y", "z"]
        return df
    raise ValueError(f"无法解析 IGMAS xyz 文件列结构：{path}")


def clean_igmas_xyz(df_raw: pd.DataFrame,
                    z_min: float = -75.0,
                    z_max: float = -1.0,
                    bad_values=(0.0, -80.0)) -> pd.DataFrame:
    """
    对 IGMAS 原始 x, y, z 进行清洗：
    - 转为浮点数，非数值变为 NaN；
    - 丢弃 z 为 0、-80 等明显无效值；
    - 丢弃 z 不在 [z_min, z_max] 范围内的点；
    - 对 (x, y) 相同的记录，z 取中位数。
    """
    df = df_raw.copy()
    for col in ["x", "y", "z"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    mask = np.isfinite(df["x"]) & np.isfinite(df["y"]) & np.isfinite(df["z"])

    if bad_values is not None:
        for v in bad_values:
            mask &= (df["z"] != v)

    mask &= (df["z"] >= z_min) & (df["z"] <= z_max)
    df = df.loc[mask].copy()

    df = df.groupby(["x", "y"], as_index=False).agg(z=("z", "median"))
    return df


def convert_xy_to_lonlat(df: pd.DataFrame,
                         lon_min: float = LON_MIN,
                         lat_min: float = LAT_MIN,
                         deg2km_lon: float = DEG2KM_LON,
                         deg2km_lat: float = DEG2KM_LAT) -> pd.DataFrame:
    """
    把 IGMAS 的 x/y (km) 线性反投影回 lon/lat（度）。
    约定：x=0 对应 lon_min，y=0 对应 lat_min。
    """
    out = df.copy()
    out["lon"] = lon_min + out["x"] / deg2km_lon
    out["lat"] = lat_min + out["y"] / deg2km_lat
    return out


def scatter_to_grid(lon, lat, val,
                    lon_min=LON_MIN, lon_max=LON_MAX,
                    lat_min=LAT_MIN, lat_max=LAT_MAX,
                    nx=GRID_NX, ny=GRID_NY):
    """
    将散点插值到规则经纬网格：
    - 线性插值得到主体
    - 对线性插值的 NaN（凸包外/稀疏区），用最近邻补齐，避免绘图留白
    """
    lon = np.asarray(lon, float)
    lat = np.asarray(lat, float)
    val = np.asarray(val, float)

    glon, glat = np.meshgrid(
        np.linspace(lon_min, lon_max, nx),
        np.linspace(lat_min, lat_max, ny)
    )

    if not _HAVE_SCIPY:
        return glon, glat, None

    pts = np.column_stack([lon, lat])
    v_lin = griddata(pts, val, (glon, glat), method="linear")
    v_near = griddata(pts, val, (glon, glat), method="nearest")
    v = np.where(np.isfinite(v_lin), v_lin, v_near)
    return glon, glat, v


def plot_igmas_moho(df_ll: pd.DataFrame, out_png: str):
    """
    画 IGMAS Moho 深度图：
    - 优先：散点→规则网格插值（无边界空缺）
    - 若无 SciPy：退回 tricontourf（可能有边界空缺）
    """
    lon = df_ll["lon"].values
    lat = df_ll["lat"].values
    z = df_ll["z"].values

    fig, ax = plt.subplots(figsize=(6, 6), constrained_layout=True)

    if _HAVE_SCIPY:
        glon, glat, gz = scatter_to_grid(lon, lat, z)
        levels = np.linspace(np.nanmin(gz), np.nanmax(gz), 41)
        im = ax.contourf(glon, glat, gz, levels=levels, cmap="RdBu_r")
        
    else:
        triang = mtri.Triangulation(lon, lat)
        levels = np.linspace(np.nanmin(z), np.nanmax(z), 41)
        im = ax.tricontourf(triang, z, levels=levels, cmap="RdBu_r")
        

    cbar = fig.colorbar(im, ax=ax, orientation="horizontal", pad=0.08)
    cbar.set_label("Moho depth (km)")

    ax.set_title("IGMAS Moho depth (km)")
    ax.set_xlabel("Longitude (°E)")
    ax.set_ylabel("Latitude (°)")
    ax.set_xlim(LON_MIN, LON_MAX)
    ax.set_ylim(LAT_MIN, LAT_MAX)
    ax.set_aspect("equal", adjustable="box")
    ax.grid(True, linestyle="--", alpha=0.3)

    fig.savefig(out_png, dpi=300)
    plt.close(fig)
    print(f"[FIG] IGMAS Moho 图已保存: {out_png}")


# ========= 主流程 =========
def main():
    in_path = Path(IGMAS_XYZ_PATH)
    if not in_path.exists():
        raise FileNotFoundError(f"找不到 IGMAS XYZ 文件: {in_path}")

    print(f"[INFO] 读取 IGMAS XYZ: {in_path}")
    df_raw = read_igmas_xyz(in_path)
    print(f"[INFO] 原始点数 = {len(df_raw)}")

    df_clean = clean_igmas_xyz(df_raw)
    print(f"[INFO] 清洗后点数 = {len(df_clean)}")

    df_ll = convert_xy_to_lonlat(df_clean)
    print(
        f"[INFO] lon ≈ [{df_ll['lon'].min():.3f}, {df_ll['lon'].max():.3f}] °E, "
        f"lat ≈ [{df_ll['lat'].min():.3f}, {df_ll['lat'].max():.3f}] °"
    )
    print(f"[INFO] LAT_REF = {LAT_REF:.3f}°, DEG2KM_LON = {DEG2KM_LON:.3f} km/deg")

    plot_igmas_moho(df_ll, OUT_FIG_PATH)


if __name__ == "__main__":
    main()


[INFO] 读取 IGMAS XYZ: E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\moho_results.csv
[INFO] 原始点数 = 430
[INFO] 清洗后点数 = 430
[INFO] lon ≈ [110.995, 116.995] °E, lat ≈ [14.545, 18.457] °
[INFO] LAT_REF = 16.500°, DEG2KM_LON = 106.621 km/deg
[FIG] IGMAS Moho 图已保存: E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\IGMAS_Moho_clean.png


In [9]:
# moho_compare_igmas_crust1.py
# -*- coding: utf-8 -*-
"""
功能：
1. 读取 IGMAS 反演的 Moho xyz（x y z，单位 km，z 为深度，向下为负）。
2. 把 x/y 转回 lon/lat（度）。
3. 在 lon/lat 点位上，从 CRUST1.0 netCDF 插值 Moho 深度 (mantle_top)。
4. 计算差值 dz = IGMAS - CRUST1.0 与 RMS。
5. 绘制差值图（散点→规则网格插值，避免边界留白）。

依赖：
- numpy, pandas, xarray, matplotlib
- scipy（可选；若无则退回 tricontourf）
"""

from pathlib import Path
import math

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

try:
    from scipy.interpolate import griddata
    _HAVE_SCIPY = True
except Exception:
    _HAVE_SCIPY = False

import matplotlib.tri as mtri


# ========== 配置区：按你自己的路径修改 ==========
IGMAS_XYZ_PATH = r"E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\moho_results.csv"
CRUST1_NC_PATH = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\CRUST1.0-rho.r0.1.nc"

OUT_IGMAS_LONLAT_CSV    = r"E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\SCS_IGMAS_Moho_lonlat.csv"
OUT_CRUST_SAMPLED_CSV   = r"E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\CRUST1_Moho_on_IGMAS_grid.csv"
OUT_FIG_PATH            = r"E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\IGMAS_vs_CRUST1_Moho_SCS.png"

# 区域范围
LON_MIN = 111.0
LON_MAX = 117.0
LAT_MIN = 14.5
LAT_MAX = 18.5

# 关键修改：用区域中纬度做经度方向换算
LAT_REF = 0.5 * (LAT_MIN + LAT_MAX)

DEG2KM_LAT = 111.2
DEG2KM_LON = DEG2KM_LAT * math.cos(math.radians(LAT_REF))

# 剔除边界宽度（单位：度）——两侧各 margin 度
LON_MARGIN_DEG = 0.5
LAT_MARGIN_DEG = 0.5

# 差值图色标最大绝对值（km）
DIFF_CLIP_KM = 60.0

# 规则网格分辨率（绘图用）
GRID_NX = 360
GRID_NY = 360


# ========== 工具函数 ==========
def read_igmas_xyz(path: Path) -> pd.DataFrame:
    try:
        df = pd.read_csv(path, sep=r"\s+", comment="#", engine="python")
        if {"x", "y", "z"}.issubset(df.columns):
            return df[["x", "y", "z"]]
        if df.shape[1] >= 3:
            df = df.iloc[:, :3].copy()
            df.columns = ["x", "y", "z"]
            return df
    except Exception:
        pass

    df = pd.read_csv(path, comment="#")
    if {"x", "y", "z"}.issubset(df.columns):
        return df[["x", "y", "z"]]
    if df.shape[1] >= 3:
        df = df.iloc[:, :3].copy()
        df.columns = ["x", "y", "z"]
        return df
    raise ValueError(f"无法解析 IGMAS xyz 文件列结构：{path}")


def clean_igmas_xyz(df_raw: pd.DataFrame,
                    z_min: float = -75.0,
                    z_max: float = -1.0,
                    bad_values=(0.0, -80.0)) -> pd.DataFrame:
    df = df_raw.copy()
    for col in ["x", "y", "z"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    mask = np.isfinite(df["x"]) & np.isfinite(df["y"]) & np.isfinite(df["z"])
    if bad_values is not None:
        for v in bad_values:
            mask &= (df["z"] != v)
    mask &= (df["z"] >= z_min) & (df["z"] <= z_max)

    df = df.loc[mask].copy()
    df = df.groupby(["x", "y"], as_index=False).agg(z=("z", "median"))
    return df


def convert_xy_to_lonlat(df: pd.DataFrame,
                         lon_min: float = LON_MIN,
                         lat_min: float = LAT_MIN,
                         deg2km_lon: float = DEG2KM_LON,
                         deg2km_lat: float = DEG2KM_LAT) -> pd.DataFrame:
    out = df.copy()
    out["lon"] = lon_min + out["x"] / deg2km_lon
    out["lat"] = lat_min + out["y"] / deg2km_lat
    return out


def sample_crust1_moho(nc_path: Path, lons, lats) -> np.ndarray:
    ds = xr.open_dataset(nc_path)
    mantle_top = ds["mantle_top"]

    lons = np.asarray(lons, dtype=np.float64)
    lats = np.asarray(lats, dtype=np.float64)

    da_sample = mantle_top.interp(
        longitude=("points", lons),
        latitude=("points", lats),
        method="linear",
    )
    return da_sample.values


def compute_rms(diff, mask=None) -> float:
    diff = np.asarray(diff, dtype=float)
    if mask is not None:
        mask = np.asarray(mask, dtype=bool)
        diff = diff[mask]
    diff = diff[np.isfinite(diff)]
    if diff.size == 0:
        return np.nan
    return float(np.sqrt(np.mean(diff ** 2)))


def scatter_to_grid(lon, lat, val,
                    lon_min=LON_MIN, lon_max=LON_MAX,
                    lat_min=LAT_MIN, lat_max=LAT_MAX,
                    nx=GRID_NX, ny=GRID_NY):
    lon = np.asarray(lon, float)
    lat = np.asarray(lat, float)
    val = np.asarray(val, float)

    glon, glat = np.meshgrid(
        np.linspace(lon_min, lon_max, nx),
        np.linspace(lat_min, lat_max, ny)
    )

    if not _HAVE_SCIPY:
        return glon, glat, None

    pts = np.column_stack([lon, lat])
    v_lin = griddata(pts, val, (glon, glat), method="linear")
    v_near = griddata(pts, val, (glon, glat), method="nearest")
    v = np.where(np.isfinite(v_lin), v_lin, v_near)
    return glon, glat, v


def plot_diff_only(df: pd.DataFrame, out_png: str, rms_all=None, rms_inner=None):
    lon = df["lon"].values
    lat = df["lat"].values
    dz = df["dz"].values

    fig, ax = plt.subplots(figsize=(7, 5), constrained_layout=True)

    # 控制色标范围
    max_abs_data = float(np.nanmax(np.abs(dz)))
    max_abs = min(max_abs_data, DIFF_CLIP_KM)
    levels = np.linspace(-max_abs, max_abs, 41)

    if _HAVE_SCIPY:
        glon, glat, gdz = scatter_to_grid(lon, lat, dz)
        im = ax.contourf(glon, glat, gdz, levels=levels, cmap="RdBu_r")
    else:
        triang = mtri.Triangulation(lon, lat)
        im = ax.tricontourf(triang, dz, levels=levels, cmap="RdBu_r")

    cbar = fig.colorbar(im, ax=ax, orientation="horizontal", pad=0.12)
    cbar.set_label("ΔDepth (km)")

    ax.set_xlim(LON_MIN, LON_MAX)
    ax.set_ylim(LAT_MIN, LAT_MAX)
    ax.set_xlabel("Longitude (°E)")
    ax.set_ylabel("Latitude (°)")
    ax.set_aspect("equal", adjustable="box")
    ax.grid(True, linestyle="--", alpha=0.3)

    lines = []
    if rms_all is not None and not np.isnan(rms_all):
        lines.append(f"RMS(all) = {rms_all:.1f} km")
    if rms_inner is not None and not np.isnan(rms_inner):
        lines.append(f"RMS(inner) = {rms_inner:.1f} km")
    if lines:
        ax.text(
            0.02, 0.98, "\n".join(lines),
            transform=ax.transAxes,
            ha="left", va="top", fontsize=10,
            bbox=dict(facecolor="white", alpha=0.25, edgecolor="none"),
        )

    ax.set_title("Moho: IGMAS − CRUST1.0", fontsize=14)
    fig.savefig(out_png, dpi=300)
    plt.close(fig)
    print(f"[FIG] 差值图已保存到: {out_png}")


# ========== 主流程 ==========
def main():
    in_path = Path(IGMAS_XYZ_PATH)
    if not in_path.exists():
        raise FileNotFoundError(f"找不到 IGMAS XYZ 文件: {in_path}")

    print(f"[INFO] 读取 IGMAS XYZ: {in_path}")
    df_raw = read_igmas_xyz(in_path)
    print(f"[INFO] 原始点数 = {len(df_raw)}")

    df = clean_igmas_xyz(df_raw)
    print(f"[INFO] 清洗后点数 = {len(df)}")

    df = convert_xy_to_lonlat(df)
    print(f"[INFO] LAT_REF = {LAT_REF:.3f}°, DEG2KM_LON = {DEG2KM_LON:.3f} km/deg")
    print(
        f"[INFO] lon ≈ [{df['lon'].min():.3f}, {df['lon'].max():.3f}] °E, "
        f"lat ≈ [{df['lat'].min():.3f}, {df['lat'].max():.3f}] °"
    )

    out_lonlat_path = Path(OUT_IGMAS_LONLAT_CSV)
    out_lonlat_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_lonlat_path, index=False)
    print(f"[OUT] IGMAS Moho (含 lon/lat) 已导出: {out_lonlat_path}")

    nc_path = Path(CRUST1_NC_PATH)
    if not nc_path.exists():
        raise FileNotFoundError(f"找不到 CRUST1.0 netCDF 文件: {nc_path}")

    print(f"[INFO] 从 CRUST1.0 采样 Moho (mantle_top): {nc_path}")
    moho_crust = sample_crust1_moho(nc_path, df["lon"].values, df["lat"].values)
    df["moho_crust"] = moho_crust

    out_crust_path = Path(OUT_CRUST_SAMPLED_CSV)
    out_crust_path.parent.mkdir(parents=True, exist_ok=True)
    df[["lon", "lat", "moho_crust"]].to_csv(out_crust_path, index=False)
    print(f"[OUT] CRUST1.0 Moho 在 IGMAS 网格上的采样结果已导出: {out_crust_path}")

    df["dz"] = df["z"] - df["moho_crust"]

    rms_all = compute_rms(df["dz"].values)

    inner_lon_min = LON_MIN + LON_MARGIN_DEG
    inner_lon_max = LON_MAX - LON_MARGIN_DEG
    inner_lat_min = LAT_MIN + LAT_MARGIN_DEG
    inner_lat_max = LAT_MAX - LAT_MARGIN_DEG

    if inner_lon_min >= inner_lon_max or inner_lat_min >= inner_lat_max:
        print("[WARN] LON_MARGIN_DEG / LAT_MARGIN_DEG 太大，内部区域为空，不计算内部 RMS。")
        rms_inner = np.nan
    else:
        mask_inner = (
            (df["lon"].values >= inner_lon_min) & (df["lon"].values <= inner_lon_max) &
            (df["lat"].values >= inner_lat_min) & (df["lat"].values <= inner_lat_max)
        )
        rms_inner = compute_rms(df["dz"].values, mask_inner)

    print(f"[RMS] 全区域 RMS(IGMAS - CRUST1.0) = {rms_all:.3f} km")
    print(f"[RMS] 剔除边界后的 RMS = {rms_inner:.3f} km")

    plot_diff_only(df, OUT_FIG_PATH, rms_all=rms_all, rms_inner=rms_inner)


if __name__ == "__main__":
    main()


[INFO] 读取 IGMAS XYZ: E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\moho_results.csv
[INFO] 原始点数 = 430
[INFO] 清洗后点数 = 430
[INFO] LAT_REF = 16.500°, DEG2KM_LON = 106.621 km/deg
[INFO] lon ≈ [110.995, 116.995] °E, lat ≈ [14.545, 18.457] °
[OUT] IGMAS Moho (含 lon/lat) 已导出: E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\SCS_IGMAS_Moho_lonlat.csv
[INFO] 从 CRUST1.0 采样 Moho (mantle_top): E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\CRUST1.0-rho.r0.1.nc
[OUT] CRUST1.0 Moho 在 IGMAS 网格上的采样结果已导出: E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\CRUST1_Moho_on_IGMAS_grid.csv
[RMS] 全区域 RMS(IGMAS - CRUST1.0) = 7.398 km
[RMS] 剔除边界后的 RMS = 8.583 km
[FIG] 差值图已保存到: E:\wjy\Gravity\SCS_Gravity\out\Outdata\igmas\IGMAS_vs_CRUST1_Moho_SCS.png


In [ ]:
# extract_moho_along_jiang.py
# -*- coding: utf-8 -*-
"""
功能：
1. 读取 Moho 离散点 CSV（IGMAS/CRUST1 等模型的 Moho 面）。
2. 使用 PyGMT.surface 将散点插值为规则格网（NetCDF/xarray.Grid）。
3. 读取 Jiang, 2024 的 PNG2010 Line01 剖面坐标。
4. 用 PyGMT.grdtrack 沿剖面坐标抽取模型 Moho 深度。
5. 计算模型与 Jiang Moho 的残差与 RMS。
6. 画出对比剖面并导出对比结果到 CSV。
"""

import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import pygmt


# ===================== 1. 路径设置 =====================
# 根据你的实际路径修改
moho_points_csv = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\Moho\CRUST1_Moho_on_IGMAS_grid_Crust3.0_UM3.3.csv"
jiang_profile_txt = r"E:\wjy\Gravity\OJP_Gravity\data\文献相关地震剖面数据\Jiang,2024,PNG2010_Line01_Moho.txt"

# 输出文件
moho_grid_nc = "E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\Moho_model_surface_0.25deg.nc"
compare_csv = "E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\Moho_compare_Jiang_PNG2010_Line01.csv"
compare_fig = "E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\Moho_compare_Jiang_PNG2010_Line01.png"


# ===================== 2. 读取 Moho 离散点并插值为格网 =====================
print(">>> 读取 Moho 散点 CSV ...")
df_moho = pd.read_csv(moho_points_csv)

# 检查列名
expected_cols = {"lon", "lat", "moho_crust"}
if not expected_cols.issubset(df_moho.columns):
    raise ValueError(f"输入文件 {moho_points_csv} 中缺少列：{expected_cols - set(df_moho.columns)}")

# 将 moho_crust 转为“向下为正”的深度（单位 km）
# 当前文件 moho_crust 大约在 [-34, -9] km，因此取负号
df_moho["moho_depth_km"] = -df_moho["moho_crust"]

# 定义插值区域和网格间距
lon_min = df_moho["lon"].min()
lon_max = df_moho["lon"].max()
lat_min = df_moho["lat"].min()
lat_max = df_moho["lat"].max()

# 适当留一点边界余量
margin = 0.05
region = [
    lon_min - margin,
    lon_max + margin,
    lat_min - margin,
    lat_max + margin,
]

# 网格间距（度）——可以根据你原始IGMAS网格分辨率设置，比如 0.25 或 0.1
spacing = 0.1

print(">>> 使用 PyGMT.surface 插值生成 Moho 格网 ...")
grid = pygmt.surface(
    data=df_moho[["lon", "lat", "moho_depth_km"]],
    region=region,
    spacing=spacing,
    tension=0.5,  # 可根据需要调整平滑程度
)

# 保存为 NetCDF，方便后续 GMT / PyGMT 使用（可选）
print(f">>> 保存 Moho 格网到 {moho_grid_nc}")
grid.to_netcdf(moho_grid_nc)


# ===================== 3. 读取 Jiang 剖面文件 =====================
print(">>> 读取 Jiang 剖面文件 ...")
jiang = pd.read_csv(jiang_profile_txt, delim_whitespace=True)

required_cols = {"Distance", "Longitude", "Latitude", "Moho_depth"}
if not required_cols.issubset(jiang.columns):
    raise ValueError(f"Jiang 剖面文件缺少列：{required_cols - set(jiang.columns)}")

# Jiang 的 Moho_depth 假定为 km、向下为正（从数值大小 ~25 km 可以看出来）
# 如果原论文定义不同，可以在这里统一符号


# ===================== 4. 使用 PyGMT.grdtrack 抽取模型 Moho =====================
print(">>> 使用 grdtrack 沿 Jiang 剖面抽取模型 Moho ...")

# 只取经纬度作为采样点
points = jiang[["Longitude", "Latitude"]]

# 用 xarray.Grid 作为输入
track = pygmt.grdtrack(
    points=points,
    grid=grid,               # 或直接用 moho_grid_nc 文件名
    newcolname="Moho_model_km",
)

# PyGMT 会返回一个 DataFrame，包含原始点列 + 新列 Moho_model_km
# 为了保证索引对齐，这里按 index 直接 join
jiang_model = jiang.join(track["Moho_model_km"])

# ===================== 5. 计算残差与 RMS =====================
print(">>> 计算模型与 Jiang Moho 的残差与 RMS ...")

# 仅在 Jiang 有 Moho_depth 的位置统计（剖面两端有 NaN）
mask_valid = ~jiang_model["Moho_depth"].isna() & ~jiang_model["Moho_model_km"].isna()

misfit = jiang_model.loc[mask_valid, "Moho_model_km"] - jiang_model.loc[mask_valid, "Moho_depth"]
rms = float(np.sqrt(np.mean(misfit**2)))

print(f"  有效点数：{mask_valid.sum()}")
print(f"  RMS(moho_model - moho_Jiang) = {rms:.3f} km")

jiang_model["Moho_misfit_km"] = jiang_model["Moho_model_km"] - jiang_model["Moho_depth"]

# 保存对比结果
print(f">>> 保存对比结果到 {compare_csv}")
jiang_model.to_csv(compare_csv, index=False)

# ===================== 6. 绘图：沿距离的 Moho 对比剖面 =====================
print(f">>> 绘制对比图并保存到 {compare_fig}")

plt.figure(figsize=(8, 5))

# Jiang Moho
plt.plot(
    jiang_model["Distance"],
    jiang_model["Moho_depth"],
    "k-",
    label="Jiang 2024 Moho (km)",
)


plt.plot(
    jiang_model["Distance"],
    jiang_model["Moho_model_km"],
    "r-",
    label="Model Moho (IGMAS) (km)",
)

plt.gca().invert_yaxis()  
plt.xlabel("Distance along profile (km)")
plt.ylabel("Moho depth (km)")
plt.title(f"Moho along PNG2010 Line01\nRMS = {rms:.3f} km")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.savefig(compare_fig, dpi=300)
plt.close()

print(">>> 完成。")


In [ ]:
# add_gladczenko_moho_to_transect.py
# -*- coding: utf-8 -*-
"""
功能：
1. 读取 OJP_Nauru_Basin.csv（经纬度点）。
2. 计算沿剖面距离 Distance_km（从图 6 左端开始累积）。
3. 读取在 WebPlotDigitizer 中 digitize 的 Moho 曲线
   （例如 OJP_Furumoto_Nauru_Moho_1976.csv 或 Gladczenko_1997_OJP_Nauru_Moho_digitized.csv）。
4. 将 digitize 的 Moho 深度插值到每个经纬度点上，得到 Moho_Gladczenko_km。
5. 输出一个带有 lon, lat, Distance_km, Moho_Gladczenko_km 的新 CSV。
"""

import numpy as np
import pandas as pd
from pyproj import Geod

# ============= 1. 读取航迹经纬度 =============
transect_csv = r"E:\wjy\Gravity\OJP_Gravity\data\文献相关地震剖面数据\data\OJP_Nauru_Basin.csv"
df = pd.read_csv(transect_csv, header=None, names=["lon", "lat"])


print(">>> OJP-Nauru transect: first and last points:")
print(df.head(3))
print(df.tail(3))

# ============= 2. 计算沿剖面距离 =============
geod = Geod(ellps="WGS84")

dist_km = np.zeros(len(df))
for i in range(1, len(df)):
    # geod.inv 返回 (fwd_azimuth, back_azimuth, distance_m)
    _, _, d = geod.inv(
        df.loc[i - 1, "lon"], df.loc[i - 1, "lat"],
        df.loc[i, "lon"],     df.loc[i, "lat"]
    )
    dist_km[i] = dist_km[i - 1] + d / 1000.0  # m -> km

df["Distance_km"] = dist_km
print(f">>> 剖面总长度 ≈ {dist_km[-1]:.1f} km")

# ============= 3. 读取 digitize 的 Moho 曲线 =============
# 这里你可以换成 Gladczenko 的 Moho 或 Furumoto 的 Moho
moho_digitized_csv = r"E:\wjy\Gravity\OJP_Gravity\data\文献相关地震剖面数据\data\OJP_Nauru_Moho_1997.csv"
ref = pd.read_csv(moho_digitized_csv)

# 情况 1：WPD 导出时有 header，列名就是 X, Y
if {"X", "Y"}.issubset(ref.columns):
    ref_dist = ref["X"].to_numpy()   # 沿剖面距离 (km)
    ref_depth = ref["Y"].to_numpy()  # Moho 深度 (km)
else:
    # 情况 2：WPD 没有 header，第一行数值被当成列名
    cols = list(ref.columns)
    if len(cols) < 2:
        raise ValueError(f"{moho_digitized_csv} 至少需要两列数据（X, Y），当前只有 {len(cols)} 列。")
    print(">>> digitize 文件原始列名（前两列）：", cols[:2])

    # 将前两列重命名为 X, Y
    ref = ref.rename(columns={cols[0]: "X", cols[1]: "Y"})
    ref_dist = ref["X"].astype(float).to_numpy()
    ref_depth = ref["Y"].astype(float).to_numpy()

print(f">>> digitize 的距离范围：{ref_dist.min():.1f} ~ {ref_dist.max():.1f} km")
print(f">>> digitize 的深度范围：{ref_depth.min():.1f} ~ {ref_depth.max():.1f} km")

# ============= 4. 距离轴对齐 & 插值 =============
# 线性缩放：让 digitize 的最大距离对齐真实航迹长度
scale = df["Distance_km"].max() / ref_dist.max()
ref_dist_scaled = ref_dist * scale
print(f">>> 距离缩放因子 scale = {scale:.4f}")

# 在每个航迹点的 Distance_km 上做 1D 插值
df["Moho_Gladczenko_km"] = np.interp(
    df["Distance_km"].to_numpy(),
    ref_dist_scaled,
    ref_depth,
    left=np.nan,
    right=np.nan,
)

# ============= 5. 输出结果 =============
out_csv = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\OJP_Nauru_Moho.csv"
df.to_csv(out_csv, index=False)
print(f">>> 已输出：{out_csv}")


In [ ]:
# compare_moho_along_nauru.py
# -*- coding: utf-8 -*-
"""
功能：
1. 读取 Moho 离散点 CSV（IGMAS/CRUST1 等模型的 Moho 面）。
2. 使用 PyGMT.surface 将散点插值为规则格网（NetCDF/xarray.Grid）。
3. 读取 OJP–Nauru 剖面的经纬度和参考 Moho 深度（Gladczenko/Furumoto）。
4. 用 PyGMT.grdtrack 沿剖面坐标抽取模型 Moho 深度。
5. 计算模型与参考 Moho 的残差与 RMS。
6. 画出对比剖面并导出对比结果到 CSV。
"""

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import pygmt

# ===================== 1. 路径设置 =====================
moho_points_csv = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\Moho\CRUST1_Moho_on_IGMAS_grid_Crust3.0_UM3.3.csv"
nauru_profile_csv = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\OJP_Lyra_Moho.csv"

moho_grid_nc = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\Moho_model_surface_0.10deg.nc"
compare_csv = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\Moho_compare_OJP_Lyra.csv"
compare_fig = r"E:\wjy\Gravity\OJP_Gravity\out\Outdata\igmas\out\IGMAS_compare\Moho_compare_OJP_Lyra.png"

# ===================== 2. 读取 Moho 散点并插值为格网 =====================
print(">>> 读取 Moho 散点 CSV ...")
df_moho = pd.read_csv(moho_points_csv)

expected_cols = {"lon", "lat", "moho_crust"}
if not expected_cols.issubset(df_moho.columns):
    raise ValueError(f"输入文件 {moho_points_csv} 中缺少列：{expected_cols - set(df_moho.columns)}")

df_moho["moho_depth_km"] = -df_moho["moho_crust"]

lon_min = df_moho["lon"].min()
lon_max = df_moho["lon"].max()
lat_min = df_moho["lat"].min()
lat_max = df_moho["lat"].max()

margin = 0.05
spacing = 0.10  # 0.1 度

# 让 region 和 spacing 匹配，避免 GMT warning
x0 = lon_min - margin
x1 = lon_max + margin
y0 = lat_min - margin
y1 = lat_max + margin

nx = int(round((x1 - x0) / spacing))
ny = int(round((y1 - y0) / spacing))
x1 = x0 + nx * spacing
y1 = y0 + ny * spacing
region = [x0, x1, y0, y1]

print(">>> 使用 PyGMT.surface 插值生成 Moho 格网 ...")
grid = pygmt.surface(
    data=df_moho[["lon", "lat", "moho_depth_km"]],
    region=region,
    spacing=spacing,
    tension=0.5,
)

print(f">>> 保存 Moho 格网到 {moho_grid_nc}")
grid.to_netcdf(moho_grid_nc)

# ===================== 3. 读取 OJP–Nauru 剖面文件 =====================
print(">>> 读取 OJP–Nauru 剖面文件 ...")
nauru = pd.read_csv(nauru_profile_csv)

required_cols = {"lon", "lat", "Distance_km", "Moho_Gladczenko_km"}
if not required_cols.issubset(nauru.columns):
    raise ValueError(f"Nauru 剖面文件缺少列：{required_cols - set(nauru.columns)}")

nauru = nauru.rename(columns={
    "lon": "Longitude",
    "lat": "Latitude",
    "Distance_km": "Distance",
    "Moho_Gladczenko_km": "Moho_depth_ref"
})

# ===================== 4. 使用 PyGMT.grdtrack 抽取模型 Moho =====================
print(">>> 使用 grdtrack 沿 OJP–Nauru 剖面抽取模型 Moho ...")
points = nauru[["Longitude", "Latitude"]]

track = pygmt.grdtrack(
    points=points,
    grid=grid,
    newcolname="Moho_model_km",
)

nauru_model = nauru.join(track["Moho_model_km"])

# ===================== 5. 计算残差与 RMS =====================
print(">>> 计算模型与参考 Moho 的残差与 RMS ...")
mask_valid = ~nauru_model["Moho_depth_ref"].isna() & ~nauru_model["Moho_model_km"].isna()

misfit = nauru_model.loc[mask_valid, "Moho_model_km"] - nauru_model.loc[mask_valid, "Moho_depth_ref"]
rms = float(np.sqrt(np.mean(misfit ** 2)))

print(f"  有效点数：{mask_valid.sum()}")
print(f"  RMS(moho_model - moho_ref) = {rms:.3f} km")

nauru_model["Moho_misfit_km"] = nauru_model["Moho_model_km"] - nauru_model["Moho_depth_ref"]

print(f">>> 保存对比结果到 {compare_csv}")
nauru_model.to_csv(compare_csv, index=False)

# ===================== 6. 绘图 =====================
print(f">>> 绘制对比图并保存到 {compare_fig}")

plt.figure(figsize=(8, 5))

plt.plot(
    nauru_model["Distance"],
    nauru_model["Moho_depth_ref"],
    "k-",
    label="Moho (Gladczenko) (km)",
)

plt.plot(
    nauru_model["Distance"],
    nauru_model["Moho_model_km"],
    "r-",
    label="Model Moho (IGMAS) (km)",
)

plt.gca().invert_yaxis()
plt.xlabel("Distance along OJP–Lyra transect (km)")
plt.ylabel("Moho depth (km)")
plt.title(f"Moho along OJP–Lyra\nRMS = {rms:.3f} km")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.savefig(compare_fig, dpi=300)
plt.close()

print(">>> 完成。")
